<a href="https://colab.research.google.com/github/veridelisi/langchain/blob/main/4)_Document_Loader_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

What you will learn:

- Document Loading a PDF
- Document Loading a CSV

## Make sure you import the two files
- state.txt
- bank.pdf

In [ ]:
!pip install -qU langchain openai tiktoken faiss-cpu pypdf chromadb nltk

In [2]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import TextLoader
from langchain.llms import OpenAI
import os
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import PyPDFLoader

OPENAI_API_KEY = "......"

llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

In [ ]:
loader = TextLoader('./sample_data/veri.txt')

documents = loader.load()

print(documents)


**chunk_size:** the maximum size of your chunks (as measured by the length function).

**chunk_overlap:** the maximum overlap between chunks. It can be nice to have some overlap to maintain some continuity between chunks (eg do a sliding window).

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=0)

texts = text_splitter.split_documents(documents)

print(texts)

[Document(page_content='İspanyolca yazıldığı gibi mi okunur ? Evet genel olarak yazıldığı gibi okunur. Google çeviriye girip hangi kelimenin nasıl okunduğunu rahatlıkla görebilirsiniz. Google çeviriye şu adresten girin ve', metadata={'source': './sample_data/veri.txt'}), Document(page_content='resim üzerinde işaretlediğim ses ikonuna tıklayın.', metadata={'source': './sample_data/veri.txt'}), Document(page_content='İspanyolca okunuşu ve yazılışı aynı mı?', metadata={'source': './sample_data/veri.txt'}), Document(page_content='Bu sitede ispanyolca kelimelerin okunuşlarını dinleyebilir miyim ? Evet web sitemiz içerisinde bulunan ispanyolca yazılı kelimelerin okunuşlarını aşağıdaki dinleme aracını ve videoları gördüğünüz', metadata={'source': './sample_data/veri.txt'}), Document(page_content='yerlerde dinleyebilirsiniz. Hadi tıklayın ve izleyen ispanyolca kelimelerin nasıl okunduğunu kendi kulağınızla dinleyin ve izleyin ! Hola, Buenos Días, Buenas Tardes, Buenas Noches', metadata={'sourc

In [13]:
os.environ["OPENAI_API_KEY"] = "......."

embeddings = OpenAIEmbeddings()

db = FAISS.from_documents(texts, embeddings)

retriever = db.as_retriever()

docs = retriever.get_relevant_documents("ispanyolca kolay mı?")

##
print(docs)

[Document(page_content='İspanyolca okunuşu ve yazılışı aynı mı?', metadata={'source': './sample_data/veri.txt'}), Document(page_content='İspanyolca öğrenmek türkler için zor mudur ? Aksine daha kolaydır. Mesela fiil çekim yapısı türkçeye benzer. İngilizcenin sıkıcılığından kurtulun artık.', metadata={'source': './sample_data/veri.txt'}), Document(page_content='Neden ispanyolca öğrenmek istiyorum ? Yahu ne bileyim. 🙂', metadata={'source': './sample_data/veri.txt'}), Document(page_content='İspanyolca okunuşu ve yazılışı aynı mı? Evet genel olarak okunuşu ve yazılışı aynıdır. Basit ispanyolca derslerimizin hepsinde videolu anlatım yapılmıştır. Tıklayın ve görün.', metadata={'source': './sample_data/veri.txt'})]


In [14]:
chain = load_summarize_chain(llm, chain_type="map_reduce")
chain.run(docs)

' Bu soru, İspanyolca kelimelerinin okunuşu ve yazılışı arasındaki farkı sorgulamaktadır. Videolu anlatımlarla desteklenen basit İspanyolca dersleri sunulmaktadır. İspanyolca öğrenmek Türkler için kolaydır çünkü fiil çekim yapısı Türkçeye benzer.'

In [15]:
loader = PyPDFLoader('./sample_data/engin_yilmaz_parayi_kim_yaratir.pdf')
pages = loader.load_and_split()
print(pages[0])

page_content='Parayı Kim Yaratır? \nDr. ENGİN YILMAZ \nAnkara Üniversitesi \nLatin Amerika Çalışmaları Araştırma ve Uygulama Merkezi \nenginyilmaz@ankara.edu.tr \nhttps://orcid.org/0000-0001-7765-430X \nGönderilme Tarihi: 14.04.2020  Kabul Tarihi: 29.04.2020 \nÖz \nTüm dünyayı etkisi altına alan Covid-19 virüsünün olumsuz ekonomik etkilerine karşı, birçok \nülke ekonomik önlemler almıştır. Bu tedbirler temel olarak, üretimden kaynaklı bir düşüşün ortaya \nçıkaracağı vahim durumu önlemeye yöneliktir. Bu ülkeler, ekonomideki harcama kanalını des -\ntekleyerek bahsi geçen durumu minimize etmeye çalışmaktadırlar. İşsiz kalanların desteklenmesi, \nfinansal piyasalardaki likidite sıkışıklıklarının giderilmesi ve üretim zincirlerinde ortaya çıkacak \nfinansal sorunların giderilmesi için ekonomik paketler açıklanmıştır. Vatandaşlar açısından, dev -\nletin nasıl kaynak bulduğu ve bu büyük harcamaları nasıl yaptığı konusu açık değildir. Aynı za -\nmanda bu konu ekonomik literatürde de somut olar

In [16]:
from langchain.indexes import VectorstoreIndexCreator
index = VectorstoreIndexCreator().from_loaders([loader])


In [17]:
query = "Makalenin özü nedir"
index.query(query)

' Makalede, paranın ne olduğu, bankalar tarafından nasıl yaratıldığı, Türkiye özelinde paranın nasıl yaratıldığı ve ABD özelinde krizler karşısında devletlerin paranın nasıl yaratıldığı konularının incelendiği ve bunların nasıl etkilediği açıklanmıştır.'

In [20]:
query = "Bankalar parayı nasıl yaratır kredi vererek mi mevduat toplayarak mı?"
index.query(query)

' Bankalar kredi vererek mevduat yaratır.'